In [ ]:
# Instalar las bibliotecas necesarias
%pip install numpy pandas scikit-learn gensim transformers torch python-docx

In [2]:
# Importar las bibliotecas
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch
from docx import Document

c:\Users\gabca\OneDrive - pascualbravo.edu.co\PROYECTO NLP\Experimentos\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Función para cargar documentos .docx desde una carpeta
def load_docx_files(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.docx'):
            doc_path = os.path.join(folder_path, filename)
            doc = Document(doc_path)
            text = ' '.join([para.text for para in doc.paragraphs])
            documents.append(text)
    return documents
# Cargar documentos desde la carpeta especificada
folder_path = './Documentos de Pruebas'  # Cambia esto a la ruta de tu carpeta
documents = load_docx_files(folder_path)

In [7]:
# 1. TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

In [8]:
# 2. Word Embeddings (Word2Vec)
tokenized_docs = [doc.split() for doc in documents]
word2vec_model = Word2Vec(tokenized_docs, vector_size=100, window=5, min_count=1, workers=4)

In [9]:
# 3. BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(doc):
    inputs = tokenizer(doc, return_tensors='pt', padding=True, truncation=True)
    outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

bert_embeddings = np.array([get_bert_embeddings(doc) for doc in documents])

c:\Users\gabca\OneDrive - pascualbravo.edu.co\PROYECTO NLP\Experimentos\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# 4. Topic Modelling (LDA)
from gensim import corpora
from gensim.models import LdaModel

dictionary = corpora.Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=10)

In [11]:
# Función para calcular la similitud del coseno
def semantic_similarity_tfidf(target_doc, other_doc):
    # Vectorizar los documentos
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([target_doc, other_doc])
    # Calcular la similitud del coseno
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

def semantic_similarity_bert(target_doc, other_doc):
    # Obtener embeddings de BERT
    target_embedding = get_bert_embeddings(target_doc)
    other_embedding = get_bert_embeddings(other_doc)
    # Calcular la similitud del coseno
    similarity = cosine_similarity(target_embedding, other_embedding)
    return similarity[0][0]


In [13]:
print(documents)

['La evolución de la inteligencia artificial  La inteligencia artificial (IA) ha experimentado un crecimiento exponencial en las últimas décadas, transformando diversos aspectos de nuestra vida cotidiana y revolucionando industrias enteras. Desde sus humildes inicios como un concepto teórico, la IA ha evolucionado hasta convertirse en una tecnología omnipresente con aplicaciones prácticas en casi todos los sectores.  El término "inteligencia artificial" fue acuñado en 1956 durante la conferencia de Dartmouth, donde científicos pioneros como John McCarthy y Marvin Minsky sentaron las bases de este campo. En sus primeras etapas, la IA se centró en la resolución de problemas y el razonamiento simbólico, con sistemas expertos que intentaban emular el proceso de toma de decisiones humano.  A medida que avanzaba la tecnología, surgieron nuevos enfoques. El aprendizaje automático, una rama de la IA, ganó prominencia en los años 80 y 90. Esta técnica permite a las máquinas aprender de los dato

In [14]:
# Documento objetivo (puedes seleccionar el índice del documento que deseas comparar)
target_index = 1  # Cambia esto al índice del documento objetivo
target_document = documents[target_index]

# Comparar el documento objetivo con todos los demás documentos
for i in range(len(documents)):
    if i != target_index:  # Evitar comparar el documento consigo mismo
        tfidf_sim = semantic_similarity_tfidf(target_document, documents[i])
        bert_sim = semantic_similarity_bert(target_document, documents[i])
        
        print(f"Comparando con Documento {i}:\n- Documento: {documents[i][:30]}...")
        print(f"Similitud TF-IDF: {tfidf_sim:.4f}")
        print(f"Similitud BERT: {bert_sim:.4f}\n")

Comparando con Documento 0:
- Documento: La evolución de la inteligenci...
Similitud TF-IDF: 0.5147
Similitud BERT: 0.9541

Comparando con Documento 2:
- Documento: Los beneficios de una vida act...
Similitud TF-IDF: 0.6147
Similitud BERT: 0.9918

